In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from scipy.sparse import load_npz, csr_matrix

In [2]:
def prob_ncm(scores, labels):
    """
    Converts Neural Network scores into Nonconformity Measures for CP.
    Assumes that scores are directly related to the probability of being active
    """
    return np.where( labels > 0, -scores, scores )

### p-Values calculation
def p_values(calibration_alphas, test_alphas, randomized=False):
    sorted_cal_alphas = sorted(calibration_alphas)
    if randomized:
        # for each test alpha, tieBreaker is the (number of calibration alphas with the same value)*(uniform RV between 0 and 1)
        tie_counts = np.searchsorted(sorted_cal_alphas,test_alphas,side='right')-np.searchsorted(sorted_cal_alphas,test_alphas)
        tie_breaker = np.random.uniform(size=len(np.atleast_1d(test_alphas)))*tie_counts
        return  (len(calibration_alphas)-(np.searchsorted(sorted_cal_alphas,test_alphas,side='right')-tie_breaker)+1)/(len(calibration_alphas)+1)
    else:
        return  (len(calibration_alphas)-np.searchsorted(sorted_cal_alphas,test_alphas)+1)/(len(calibration_alphas)+1)

# Mondrian Inductive Conformal Predictor
def micp(calibration_alphas,calibration_labels,test_alphas_0,test_alphas_1,randomized=False):
    """
    Mondrian Inductive Conformal Predictor
    Parameters:
    calibration_alphas: 1d array of Nonconformity Measures for the calibration examples
    calibration_labels: 1d array of labels for the calibration examples - ideally 0/1 or -1/+1,
                        but negative/positive values also accepted
    test_alpha_0: 1d array of NCMs for the test examples, assuming 0 as label
    test_alpha_1: 1d array of NCMs for the test examples, assuming 1 as label
    Returns:
    p0,p1 : pair of arrays containing the p-values for label 0 and label 1
    """
    if not len(calibration_labels)==len(calibration_alphas):
        raise ValueError("calibration_labels and calibration alphas must have the same size")
    
    if not len(np.atleast_1d(test_alphas_0))==len(np.atleast_1d(test_alphas_1)):
        raise ValueError("test_alphas_0 and test_alphas_1 must have the same size")
    
    p_0 = p_values(calibration_alphas[calibration_labels<=0],
                   test_alphas_0,
                   randomized)
    p_1 = p_values(calibration_alphas[calibration_labels>0],
                   test_alphas_1,
                   randomized)
    return p_0,p_1

# function to predict label from p0 and p1
def cp_label_predictor(p0, p1, eps):
    # Active: p1 > ϵ and p0 ≤ ϵ
    # Inactive: p0 > ϵ and p1 ≤ ϵ
    # Uncertain (Both): p1 > ϵ and p0 > ϵ
    # Empty (None): p1 ≤ ϵ and p0 ≤ ϵ
    if p1 > eps and p0 <= eps:
        return 1
    elif p0 > eps and p1 <= eps:
        return 0
    elif p0 > eps and p1 > eps:
        return 'uncertain both'
    elif p0 <= eps and p1 <= eps:
        # return 'empty'
        # it should actually return 'empty', but to avoid a confusion for people
        return 'uncertain none'

In [3]:
# CONFIG 

eps = 0.05
fold_va = 2
path = '../../data/Mellody_tuner/images/output_images/pseudolabels'
fva_preds = '../../Classification_no_aux/predictions/01_cls/cls/pred_images-class.npy'
path_folds = path+'/matrices/cls/cls_T11_fold_vector.npy'
path_labels = path+'/matrices/cls/cls_T10_y.npz'
path_sn = path+'/results_tmp/folding/T2_folds.csv'
path_t5 = path+'/mapping_table/T5.csv' 
path_t6_cont = path+'/results/T10c_cont.csv'

In [4]:
folds = np.load(path_folds,allow_pickle=True)
labels = load_npz(path_labels).tocsr()
preds_fva = np.load(fva_preds,allow_pickle=True).item()

In [5]:
labels.shape

(10077, 285)

In [6]:
sn = pd.read_csv(path_sn)

In [7]:
preds_fva

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 184789 stored elements in Compressed Sparse Row format>

In [8]:
sn_fold2 = sn.query('fold_id == 2')
sn_scaffolds = sn_fold2.groupby(by='sn_smiles').count()['input_compound_id'].sort_values(ascending=False)
sn_scaffolds

sn_smiles
c1ccc2[nH]ccc2c1                          70
O=C1NCCCOCc2ccccc2-c2c1[nH]c1ccccc21      52
O=S1(=O)NCCCOc2cc(C#Cc3ccccc3)ccc21       35
c1ccc2ccccc2c1                            25
O=S(=O)(Nc1ccccc1)c1ccccc1                21
                                          ..
O=C1NC(=O)C(=Cc2c[nH]c3ccccc23)C(=O)N1     1
O=C1N=CC(C(=O)OCc2ccc3c(c2)OCO3)CN1        1
O=C1N=CC(C(=O)OC2CCCCC2)C(c2ccccc2)N1      1
O=C1N=CC(=Cc2ccccc2)S1                     1
c1nonc1-c1n[nH]c(-c2cnon2)n1               1
Name: input_compound_id, Length: 1209, dtype: int64

In [9]:
len(sn_scaffolds)

1209

In [10]:
np.tile([0,1],reps=len(sn_scaffolds)//2).shape

(1208,)

In [11]:
np.append(np.tile([0,1],reps=len(sn_scaffolds)//2),np.array([1]))

array([0, 1, 0, ..., 0, 1, 1])

In [12]:
sn_map = sn_scaffolds.reset_index().drop(columns='input_compound_id')
sn_map['fold_split'] = np.append(np.tile([0,1],reps=len(sn_scaffolds)//2),np.array([1])) # ensuring similar size of both groups

In [13]:
sn_mgd = pd.merge(
    sn_fold2
    ,sn_map
    ,how='inner'
    ,on='sn_smiles'
)

In [14]:
assert len(sn_mgd) == len(sn_fold2)

In [15]:
len(sn_mgd)

2159

In [16]:
# link to the cdi 

In [17]:
t5 = pd.read_csv(path_t5)
t6_cont = pd.read_csv(path_t6_cont)

In [18]:
df_mgd = pd.merge(
        pd.merge(
            t5
            ,t6_cont
            ,how='inner'
            ,on='descriptor_vector_id'
        ), 
        sn_mgd
        ,how='inner'
        ,on='input_compound_id'
)

In [19]:
df_mgd.shape

(37584, 24)

In [20]:
df_mgd.columns

Index(['input_compound_id', 'fold_id_x', 'descriptor_vector_id',
       'classification_task_id', 'fold_id_y', 'input_assay_id',
       'standard_qualifier', 'standard_value', 'threshold', 'class_label',
       'is_active', 'is_inactive', 'cont_classification_task_id',
       'cont_descriptor_vector_id', 'smiles', 'canonical_smiles', 'success',
       'error_message', 'fp_feat', 'fp_val', 'murcko_smiles', 'sn_smiles',
       'fold_id', 'fold_split'],
      dtype='object')

In [21]:
# half of fold2 will be used to fit the CP, 
# half of fold2 will be used to evaluate the CP

In [22]:
real_cdvi = pd.DataFrame(
    sorted(t6_cont['cont_descriptor_vector_id'].drop_duplicates())
)[0].to_dict()# .reset_index()
real_cdvi = {v:k for k,v in real_cdvi.items()}
real_cdvi

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [23]:
df_mgd['real_cont_descriptor_vector_id'] = df_mgd['cont_descriptor_vector_id'].map(real_cdvi)

In [24]:
cdvi_fit = np.array(list(set(df_mgd.query('fold_split == 0')['real_cont_descriptor_vector_id'])))
cdvi_eval = np.array(list(set(df_mgd.query('fold_split == 1')['real_cont_descriptor_vector_id'])))

In [25]:
cdvi_fit.shape

(1034,)

In [26]:
cdvi_eval.shape

(992,)

In [27]:
preds_fva.nonzero()

(array([    0,     0,     0, ..., 10076, 10076, 10076], dtype=int32),
 array([ 63,  64,  65, ..., 195, 220, 252], dtype=int32))

In [28]:
preds_fva

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 184789 stored elements in Compressed Sparse Row format>

In [29]:
preds_fva

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 184789 stored elements in Compressed Sparse Row format>

In [30]:
######## CP stuff ########

In [45]:
e_inacts = []
e_acts = []
val_inacts = []
val_acts = []
lit_val_inacts = []
lit_val_acts = []
unis = []
idxs = []
n_acts = []
n_inacts = []
ncms_fva_fit_dict = {}
labels_fva_fit_dict = {}


for col in tqdm(list(np.unique(preds_fva.nonzero()[1]))):
    try: 
        row_idx_preds_fit = np.intersect1d(
            preds_fva[:,col].nonzero()[0]
            ,cdvi_fit
        )
        row_idx_preds_eval = np.intersect1d(
            preds_fva[:,col].nonzero()[0]
            ,cdvi_eval
        )
        preds_fva_col = preds_fva[row_idx_preds_fit,col].toarray().squeeze()
        preds_fte_col = preds_fva[row_idx_preds_eval,col].toarray().squeeze()
        row_idx_labels_fit = np.intersect1d(
            labels[:,col].nonzero()[0]
            ,cdvi_fit
        )
        row_idx_labels_eval = np.intersect1d(
            labels[:,col].nonzero()[0]
            ,cdvi_eval
        )
        
        labels_fva_col = labels[row_idx_labels_fit,col].toarray().squeeze()
        labels_fva_col = np.where(labels_fva_col == -1,0,1)
        labels_fte_col = labels[row_idx_labels_eval,col].toarray().squeeze()
        labels_fte_col = np.where(labels_fte_col == -1,0,1)
        


        ncms_fva = prob_ncm(preds_fva_col, labels_fva_col)
        ncms_fva_fit_dict[str(col)] = ncms_fva.tolist()  # use tolist() to avoid difficulties with the serialisation
        labels_fva_fit_dict[str(col)] = labels_fva_col.tolist() # use tolist() to avoid difficulties with the serialisation
        #ncms_test_0 = prob_ncm(preds_fte_col, labels_fte_col)
        #ncms_test_1 = prob_ncm(preds_fte_col, labels_fte_col)
        ncms_test_0 = prob_ncm(preds_fte_col, np.repeat(0.,len(preds_fte_col)))
        ncms_test_1 = prob_ncm(preds_fte_col, np.repeat(1.,len(preds_fte_col)))

        p0, p1 = micp(ncms_fva,labels_fva_col,ncms_test_0,ncms_test_1,randomized=False)

        cp_test = [cp_label_predictor(pe0, pe1, eps) for pe0, pe1 in zip(p0,p1)]
        certain_idcs = np.where((np.array(cp_test) == '0') | (np.array(cp_test) == '1'))[0]
        idx_uncertain_none = np.where([e == 'uncertain none' for e in cp_test])[0]
        idx_uncertain_both = np.where([e == 'uncertain both' for e in cp_test])[0]
        idx_inact = np.where(labels_fte_col == 0)[0]
        idx_inact_certain = np.intersect1d(idx_inact,certain_idcs)
        idx_inact_both = np.intersect1d(idx_inact,idx_uncertain_both)
        idx_act = np.where(labels_fte_col == 1)[0]
        idx_act_certain = np.intersect1d(idx_act,certain_idcs)
        idx_act_both = np.intersect1d(idx_act,idx_uncertain_both)

        # efficiency 
        efficiency_inact = len(idx_inact_certain) / len(idx_inact)
        efficiency_act = len(idx_act_certain) / len(idx_act)

        # validity 
        validity_inact = \
             np.sum(np.array(cp_test)[idx_inact_certain] == labels_fte_col[idx_inact_certain].astype(str)) / \
             len(np.array(cp_test)[idx_inact_certain])
        validity_act = \
            np.sum(np.array(cp_test)[idx_act_certain] == labels_fte_col[idx_act_certain].astype(str)) / \
            len(np.array(cp_test)[idx_act_certain])

        # literature validity 
        literature_validity_inact = \
             (np.sum(np.array(cp_test)[idx_inact_certain] == labels_fte_col[idx_inact_certain].astype(str)) \
             + len(idx_inact_both)) / \
             len(idx_inact)
        literature_validity_act = \
            (np.sum(np.array(cp_test)[idx_act_certain] == labels_fte_col[idx_act_certain].astype(str)) \
            + len(idx_act_both)) / \
            len(idx_act)


        uni = np.unique(cp_test)

        e_inacts.append(efficiency_inact)
        e_acts.append(efficiency_act)
        val_inacts.append(validity_inact)
        val_acts.append(validity_act)
        lit_val_inacts.append(literature_validity_inact)
        lit_val_acts.append(literature_validity_act)
        unis.append(str(list(uni)))
        idxs.append(col)
        n_acts.append(len(idx_act))
        n_inacts.append(len(idx_inact))

    except Exception as e:
        print(e)
        


  0%|          | 0/285 [00:00<?, ?it/s]/home/rama.jabal/.conda/envs/melloddy_tuner_regression/lib/python3.6/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in long_scalars
/home/rama.jabal/.conda/envs/melloddy_tuner_regression/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in long_scalars
 36%|███▌      | 102/285 [00:00<00:01, 178.30it/s]

division by zero


 88%|████████▊ | 251/285 [00:01<00:00, 187.48it/s]

division by zero
division by zero


100%|██████████| 285/285 [00:01<00:00, 186.14it/s]


In [46]:
cdvi_eval

array([ 4097,  2056,  4105,  2062,  2063,    21,  4117,  4119,  2075,
          29,  6180,  4133,  6182,  8230,  8232,  8231,    45,  6191,
        8240,  4143,  6194,  8245,  2108,  4159,  4165,    70,  8265,
        8266,  2124,  2125,  8269,    79,  1237,  6226,  6227,  8276,
        8277,  2134,  1238,  4184,  6233,    90,  2139,  1239,  1240,
        1241,  4199,  2152,   104,  2154,  1243,  8303,  8304,  8302,
        4210,  1244,   113,   110,  4214,  4215,  2170,  2175,  6273,
        4227,  8324,  4229,   132,  6280,  3284,  2190,   145,   148,
        3286,  2200,   155,  8355,   164,  2212,  6139,  4264,   167,
        4267,  8365,   173,  6319,   179,   180,  6326,  6328,   194,
        8400,  8402,   210,  8406,   214,  6359,  2265,  2266,   215,
        6368,  4321,  4324,  4327,  4331,   236,  8432,  8433,  8434,
        8435,  2304,  8449,  4354,  4355,  2308,  4356,  8459,  2318,
        2322,  2323,  8478,  6435,  4389,   296,   297,  8490,  4394,
         303,   309,

array([  20,  195,  666,  763, 1387, 1395, 1506, 1547, 1749, 1782, 1820,
       1907, 1991, 2086, 2666, 2867, 3145, 4014, 4051, 4052, 4087, 4163,
       4305, 4664, 4913, 5029, 5237, 5502, 5626, 5792, 5808, 6032, 6396,
       6909, 6961, 7162, 7231, 7583, 7693, 7819, 8000, 8099, 8142, 8285,
       8319, 8474, 8771, 9225, 9794, 9817, 9909, 9943, 9944, 9998],
      dtype=int32)

In [47]:
# storing the inputs to the micp() function in order to obtain the CP labels for the inference predictions
import json 
with open('./cp/ncms_fva_fit_dict.json', 'w') as fp:
	json.dump(ncms_fva_fit_dict, fp)
with open('./cp/labels_fva_dict.json', 'w') as fp:
	json.dump(labels_fva_fit_dict, fp)


In [48]:
df = pd.DataFrame({
    'n_inactives_eval':n_inacts
    ,'n_actives_eval':n_acts
    ,'efficiency_0' : e_inacts
    ,'efficiency_1':e_acts
    ,'validity_0':val_inacts
    ,'validity_1':val_acts
    ,'literature_validity_0':lit_val_inacts
    ,'literature_validity_1':lit_val_acts
    ,'valuess':unis
    ,'index':idxs
}).to_csv('./cp/summary_eps_' + str(eps) + '.csv', index=False)

In [49]:
pd.DataFrame({
    'n_inactives_eval':n_inacts
    ,'n_actives_eval':n_acts
    ,'efficiency_0' : e_inacts
    ,'efficiency_1':e_acts
    ,'validity_0':val_inacts
    ,'validity_1':val_acts
    ,'literature_validity_0':lit_val_inacts
    ,'literature_validity_1':lit_val_acts
    ,'valuess':unis
    ,'index':idxs
})

,n_inactives_eval,n_actives_eval,efficiency_0,efficiency_1,validity_0,validity_1,literature_validity_0,literature_validity_1,valuess,index
0,10,4,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],0
1,9,7,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],1
2,9,6,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],2
3,11,4,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],3
4,12,7,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],4
...,...,...,...,...,...,...,...,...,...,...
277,59,26,0.033898,0.076923,0.0,1.0,0.966102,1.0,"['1', 'uncertain both']",280
278,7,13,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],281
279,6,3,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],282
280,3,9,0.000000,0.000000,NaN,NaN,1.000000,1.0,['uncertain both'],283
